<a href="https://colab.research.google.com/github/allfed/CropOpt/blob/standardize/notebooks/yield_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup (only run this section if using Colab)

In [ ]:
# We must clone the code repo into your Colab environment in order to use it.
# To do this we must first connect your GitHub account via the command line.
# Run this block, and hit Enter on each prompt, not entering any text.
# This generates an SSH key which you'll add to your GitHub account.
!ssh-keygen -t rsa -b 4096

In [ ]:
|!ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts

In [ ]:
# This line prints the key. Copy the key to your clipboard.
!cat /root/.ssh/id_rsa.pub

In [ ]:
# Now, navigate to https://github.com/settings/keys and click "New SSH key"
# Copy paste the above into the "Key" field, and give it a title, e.g. "Colab key",
# then save the key. Once done, test using this line:
!ssh -T git@github.com

In [ ]:
## Now we can clone the repo using SSH
!rm -rf CropOpt/
!git clone -b standardize git@github.com:allfed/CropOpt.git

In [ ]:
# This adds the Python files in our CropOpt repo to our path so we can import
# them

import os
import sys

module_path = os.path.abspath(os.path.join('./CropOpt'))
if module_path not in sys.path:
    sys.path.append(module_path) 

In [ ]:
# Now we'll install other dependencies from our Poetry file. We do this by first
# installing the toml package so we can read the pyproject.toml file
!pip install toml

In [ ]:
# We then load the file and read our required packages from it
import toml
config = toml.load("CropOpt/pyproject.toml")
pip_packages = config["tool"]["poetry"]["dependencies"]

In [ ]:
# Finally, we install the packages

import subprocess
import sys

def install_pip_package(package, version):
    subprocess.check_call([sys.executable, "-m", "pip", "install", f"{package}=={version}"])

for package, version in pip_packages.items():
    if package == "python":
        continue
    install_pip_package(package, version[1:])

# Yield calculation demo

The purpose of this file is to generate yields for spring wheat and spring barley, along with nutrition, and spits out a couple plots and a csv file at the end of the file. I can't emphasize enough that you should take a look at Params.ods  to understand the inputs to the model. Params.ods is a spreadsheet, you can open it with excel or open office. Also reach out to morgan@allfed.info if you have questions about this code.

First, let's import a few dependencies. 
 * Params: Custom script which imports parameters from Params.ods file
     * Params.ods is as spreadsheet file located here: https://github.com/allfed/CropOpt/Params.ods
     * based on Adin's collaboration document here: https://docs.google.com/spreadsheets/d/1Mh7bmZYHypN7bSmejCyu4hkbolQ3MRANc3LhflwRZoA/edit#gid=0
 * Plotter: utility for displaying geopandas maps and plots
 * pandas, geopandas: Data analysis and GIS packages
     * some basic things you can do with pandas: https://pandas.pydata.org/docs/getting_started/intro_tutorials/03_subset_data.html
     * some basic things you can do with geopandas: https://geopandas.org/getting_started/introduction.html
     
 * OutdoorGrowth: Custom class which contains several useful functions for outdoor growth calculations
     * if you want to understand what the model is doing to calculate yields, you need to read through OutdoorGrowth code, located here:https://github.com/allfed/CropOpt/tree/master/Modules

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from src import params
from src.plotter import Plotter
from src import outdoor_growth
from src.outdoor_growth import OutdoorGrowth
import pandas as pd
import geopandas as gpd

Now, let's import a bunch of previously created geopandas datasets

In [ ]:
params.importAll()

# NOTE - "CropOpt" is the correct suffix when in Colab, otherwise change to "../"

#total solar flux at surface , W/m^2
fsns=pd.read_pickle("CropOpt/" + params.geopandasDataDir + 'FSNS.pkl')

#surface temperature, K
ts=pd.read_pickle("CropOpt/" + params.geopandasDataDir+'TS.pkl')

#Large-scale (stable) precipitation rate, including ice precipitate but not 
#snow, m/s
precl=pd.read_pickle("CropOpt/" + params.geopandasDataDir+'PRECL.pkl')

#absolute humidity at surface, kg/kg
q=pd.read_pickle("CropOpt/" + params.geopandasDataDir+'Q.pkl')

#crop area, Ha
growArea=pd.read_pickle("CropOpt/" + params.geopandasDataDir+'CropGrowHectares.pkl')

Taking a look at sun, temperature, and rain, we see latitude and longitude, each month after the winter is defined in a column, and the cell geometry is displayed (in this case, a 2 degree by 2 degree square). The first few cells are all at the south pole, so it's unsurprising that they show strange values.

The default months in the Params.ods are May year 5- April year 6 (first and second year of simulation). 

The average temperature coefficient hottest consecutive months required to grow each crop in each region are used to estimate temperature and rainfall growth parameters.

There's an issue with the import for april of year six, for temperature. 

In [ ]:
fsns.head() #sun, W/m^2

In [ ]:
ts.head() #temp, kelvin

In [ ]:
precl.head() #rain, average m/s over the month

The grow area is shown below using hectares. Note it is only calculated for one month as it's assumed to be unchanging for now.

In [ ]:
growArea.head()

In [ ]:
q.head()

Let's also look at the values for some of the Params.ods variable values. It's important to be able to modify these if you want to be able to improve the cropOpt model. Make sure to restart the Jupyter kernel to reimport changes to params variables. The code below is looping through all the cells and all the months, so it might take a few minutes, depending on your machine.

Again, there appear to be issues with april of year 6 (second year of winter), and also possibly september year 5 (first year of winter).

In [ ]:
allMonths=params.allMonths

#initialize object of outdoor growth class
outdoorGrowth=OutdoorGrowth()

In [ ]:
plotYields = True
yields = outdoorGrowth.estimateYields(ts,precl,growArea,plotYields)

Plots above are saved here: https://github.com/allfed/CropOpt/tree/master/Data/Figures

Below, we generate the remaining plots of the figures for all months of the simulation. This will take several minutes to run. To disable some of the plots, you can set the booleans plotGrowArea,plotsun,plotTemp,plotRain,or plotHum to false.

In [ ]:
plotSun=True
plotTemp=True
plotRain=True
plotHum=True

print('Producing Sun Figures')
for month in allMonths:
	title="Solar Flux at Surface, month "+month
	legendlabel="Solar Flux (W/m^2)"
	fn="SolarFlux"+month
	Plotter.plotMap(fsns,month,title,legendlabel,fn,plotSun)

print('Producing Temp Figures')
for month in allMonths:
	title="Surface (radiative) Temperature, month "+month
	legendlabel="Temperature (K)"
	fn="SurfaceTemp"+month
	Plotter.plotMap(ts,month,title,legendlabel,fn,plotTemp)

print('Producing Rain Figures')
for month in allMonths:
	title="Precipitation "+month
	legendlabel="Precipitation (m/s)"
	fn="Precipitation"+month
	Plotter.plotMap(precl,month,title,legendlabel,fn,plotRain)

print('Producing Humidity Figures')
for month in allMonths:
	title="Humidity "+month
	legendlabel="Humidity (kg/kg)"
	fn="Humidity"+month
	Plotter.plotMap(q,month,title,legendlabel,fn,plotHum)


Plots above are saved here: https://github.com/allfed/CropOpt/tree/master/Data/Figures

If you're interested in reproducing a CSV of many of the variables, you can call the function below:

In [ ]:
outdoorGrowth.saveLandSunHumRainCSV(ts,q,fsns,precl,growArea)